# Running LSPI on the UR5

In [1]:
from __future__ import print_function
import rosgym
import gym
import numpy as np
import os
import rospy
import imp
import random
import requests
import time
import yaml
from os import path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append(os.path.dirname(os.getcwd()) + "/rl_agents/python")

In [4]:
import basic_agents
from geometry_msgs.msg import PointStamped
from visualization_msgs.msg import Marker

In [19]:
def make_environment(robot_conf_file, planning_group, num_joints, z_limit,
                     x_limits, y_limits, **kwargs):
    """
    Creates an OpenAI Gym environment corresponding to the supplied robot
    configuration file, and configuration options.

    :return: rosgym environment
    """

    env = rosgym.make_randomgoal_robot_env(
        "ur5_config_random_goal", robot_conf_file, planning_group, num_joints,
        z_limit, x_limits[0], x_limits[1], y_limits[0], y_limits[1])
    return env

class SimpleConfig(object):
    def __init__(self, config, defaults):
        self._config = config
        self._defaults = defaults

    def __getitem__(self, key):
        return self._config.get(key, self._defaults.get(key))

In [6]:
defaults = {
    "robot_conf_file":
    "/home/cannon/rl_wksp/src/rosgym/src/rosgym/ur5_config_random_goal.py",
    "planning_group":
    "manipulator",
    "num_joints":
    6,
    "z_limit":
    2.0,
    "x_min":
    -2.0,
    "x_max":
    2.0,
    "y_min":
    -2.0,
    "y_max":
    2.0,
    "num_tests":
    10,
    "num_episodes":
    100,
}

In [7]:
sc = SimpleConfig({}, defaults)

# make our random agent
print("Attempting to make the rosgym environment...")
env = make_environment(sc["robot_conf_file"], sc["planning_group"],
                       sc["num_joints"], sc["z_limit"], [sc["x_min"], sc["x_max"]],
                       [sc["y_min"], sc["y_max"]])
print("Made the rosgym environment...")

Attempting to make the rosgym environment...


Connecting to control group manipulator
Found move groups: ['endeffector', 'forearm', 'manipulator', 'shoulder', 'upper_arm', 'wrist_1', 'wrist_2']
Found planning scene: <moveit_commander.planning_scene_interface.PlanningSceneInterface object at 0x7f157ba9c250>


Connecting reset handler
Finished connecting wt_commander
Set up MoveGroupInterface
[INFO] [1551987581.427874, 156.731000]: Waiting for get_planning_scene
Set up PlanningSceneInterface
Initialized SimpleActionClient


Finished connecting reset handler
Waiting for sensor /joint_states
Got initial data for sensor /joint_states
RobotAgent_UR5 has been initialized.
Finished initializing robot
Initialized sensors
Connecting to control group manipulator


Finished waiting


Initialized move groups and joints
Initialized service proxies
Getting new goal
Sampling new goal
Goal is: [0.1463327426866397, -0.5179436805265596, 0.8069362178323958]


Finding new goal pose
Finding new goal pose
Found goal pose: x: 0.146332742687
y: -0.517943680527
z: 0.806936217832
Made the rosgym environment...


## generating sample data

In [8]:
from lspi.policy import RandomPolicy
from lspi.util import generate_sample_data

rp = RandomPolicy(env)
data = generate_sample_data(1e3, env, rp)

Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


Sampling new goal


Finding new goal pose
Finding new goal pose
Finding new goal pose
Finding new goal pose
Finding new goal pose
Found goal pose: x: 0.185095170431
y: 0.154141808811
z: 0.722175848456
New target pose: [0.18509517043120505, 0.1541418088114775, 0.7221758484559793]


In [9]:
from lspi.basis import DiscreteRBFTupleBasis
from lspi.util import BoxAnchorGen

In [10]:
anchor_list = [
    BoxAnchorGen(n=10, space=space)
    for space in env.observation_space.spaces
]

### discretize the action space

In [17]:
from lspi.util import random_discretization
from lspi.optim import LSTDQ
from lspi.policy import DiscreteActionBasisPolicy

In [13]:
n = 30
das = gym.spaces.Discrete(n)
actions = random_discretization(env.action_space, n)

In [18]:
basis = DiscreteRBFTupleBasis(anchor_list, das, env.observation_space)
lstdq = LSTDQ(basis, discount=0.7)
dabp = DiscreteActionBasisPolicy(das, basis, np.zeros(basis.rank))

In [ ]:
print("Testing random agent: {}".format(agent))

for test in xrange(sc["num_tests"]):

    print("Test #: {}".format(test))
    state = agent.reset()

    actions = []

    for episode in xrange(sc["num_episodes"]):

        print("Episode #: {}".format(episode))
        try:
            state, action, reward, next_state, done = agent.step(
                test, episode, testing=True)
            actions.append(action)
        except Exception as err:
            print(err)

        # state, action, reward, next_state, done = agent.step(i, episode)
        if reward > 0:
            success = True

        if done:
            break

    print("Test actions: {}".format(actions))